In [1]:
import pandas as pd

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/buildweek-kickstarter/data-science/main/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

In [5]:
import pandas as pd

df = pd.read_csv(DATA_PATH+'kickstarter%20-%20complete.csv?token=AOY4UTD6LBU3HG2UMUFMDYK7NE2DE')

print(df.shape)
df

(999, 13)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 0:20:50,45000,2013-01-12 0:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 4:24:11,5000,2012-03-17 3:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 1:00:00,19500,2015-07-04 8:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1005848496,The 1st Motion Tracking DIY Smart Home Securit...,Gadgets,Technology,USD,2016-12-31 3:54:32,25000,2016-11-01 2:54:32,58978,live,397,US,7235
995,1005862247,Veterans,Photography,Photography,USD,2012-08-15 6:00:00,5000,2012-07-09 5:39:06,5637,successful,87,US,5637
996,1005863674,MY VERY FIRST KICKSTARTER,Movie Theaters,Film & Video,USD,2014-08-12 20:08:35,6500,2014-07-13 20:08:35,27,failed,2,US,27
997,1005870420,This Song Is About You,Music,Music,USD,2014-07-11 23:35:00,40000,2014-06-11 23:35:00,0,failed,0,US,0


In [38]:
print(df.columns)

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged'],
      dtype='object')


In [59]:
# Feature selection
df1 = df.filter(['main_category', 'currency', 'goal', 'pledged', 'backers', 'state'], axis=1)

df1

,main_category,currency,goal,pledged,backers,state
0,Publishing,GBP,1000,0,0,failed
1,Film & Video,USD,45000,220,3,failed
2,Music,USD,5000,1,1,failed
3,Film & Video,USD,19500,1283,14,canceled
4,Food,USD,50000,52375,224,successful
...,...,...,...,...,...,...
994,Technology,USD,25000,58978,397,live
995,Photography,USD,5000,5637,87,successful
996,Film & Video,USD,6500,27,2,failed
997,Music,USD,40000,0,0,failed


In [61]:
df1['main_category'].value_counts()

Film & Video    191
Music           152
Art              94
Publishing       90
Games            87
Design           73
Technology       68
Food             67
Fashion          50
Photography      32
Comics           29
Theater          26
Crafts           17
Dance            14
Journalism        9
Name: main_category, dtype: int64

In [62]:
df1['currency'].value_counts()

USD    805
GBP     96
CAD     36
EUR     31
AUD     18
NOK      4
SEK      3
MXN      2
NZD      1
HKD      1
DKK      1
CHF      1
Name: currency, dtype: int64

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Read train_features.csv
train = pd.read_csv(DATA_PATH+'kickstarter%20-%20training.csv?token=AOY4UTCQ23SWGVQ73Y43J7S7NEY3I')
                
# Read test_features.csv
test = pd.read_csv(DATA_PATH+'kickstarter%20-%20testing.csv?token=AOY4UTGO42NX432LRSU7NPS7NEY4Y')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20) 

In [42]:
# Arrange data into X features matrix and y target vector
features = ['category', 'main_category', 'currency', 'goal', 'pledged', 'backers']
target = 'state'
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [43]:
# Check shape before encoding
X_train.shape, X_val.shape, X_test.shape

((399, 6), (100, 6), (500, 6))

In [46]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val = encoder.transform(X_val)
X_test = encoder.transform(X_test)

In [47]:
# Check shape after encoding
X_train.shape, X_val.shape, X_test.shape

((399, 537), (100, 537), (500, 537))

In [49]:
X_train.head()

,category_Action,category_Product Design,category_Indie Rock,category_Apparel,category_Painting,category_Gadgets,category_Theater,category_Journalism,category_Comics,category_Farms,category_Design,category_Tabletop Games,category_Crafts,category_Documentary,category_Children's Books,category_Photography,category_Music,category_Nonfiction,category_People,category_Web,category_Flight,category_Hip-Hop,category_Food,category_Shorts,category_Playing Cards,category_Fiction,category_Movie Theaters,category_Publishing,category_Film & Video,category_Drinks,category_DIY Electronics,category_Public Art,category_Fashion,category_Restaurants,category_Fine Art,category_Art,category_Webseries,category_Events,category_Stationery,category_DIY,...,pledged_218,pledged_13260,"pledged_2,252.99",pledged_5487,"pledged_15,040.33",pledged_41759,pledged_25816,pledged_1315,pledged_568,pledged_4600,pledged_4045,pledged_3585,pledged_2535,pledged_23079,pledged_55,pledged_1620,"pledged_3,863.37",pledged_805,pledged_895,pledged_45,pledged_124998,pledged_310,pledged_1745,pledged_2028,pledged_935,pledged_16998,pledged_293,pledged_250,pledged_7453,pledged_17771,pledged_4813,pledged_10050,pledged_501,pledged_8422,"pledged_91,972.50",pledged_1185,pledged_1700,pledged_701.66,pledged_2321,backers
140,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
472,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
310,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80
384,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
473,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19


In [48]:
# Determine majority class

target = 'state'
y_train = train[target]
y_train.value_counts(normalize=True)

failed       1
successful   0
canceled     0
live         0
suspended    0
undefined    0
Name: state, dtype: float64

In [52]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)
y_pred[:5]

['failed', 'failed', 'failed', 'failed', 'failed']

In [54]:
from sklearn.metrics import accuracy_score

# Training accuracy of predicting majority class baseline
accuracy_score(y_train, y_pred)

0.5112781954887218

In [55]:
# Calculate for our validation as well
y_val = val[target]
y_pred = [majority_class] * len(y_val)
accuracy_score(y_val, y_pred)

0.47

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train, y_train)
print('Validation Accuracy', log_reg.score(X_val, y_val))